In [1]:
! pip install transformers

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
%matplotlib inline

In [3]:
from transformers import BertTokenizer, TFBertModel
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam


In [4]:
! pip3 install keras --upgrade

In [21]:
df = pd.read_csv('./Ethos_Dataset_Multi_Label.csv',sep=';')

In [6]:
df

,comment,violence,directed_vs_generalized,gender,race,national_origin,disability,religion,sexual_orientation
0,You should know women's sports are a joke,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,You look like Sloth with deeper Down’s syndrome,0.000000,0.833333,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
2,You look like Russian and speak like Indian. B...,1.000000,0.500000,0.142857,0.142857,0.714286,0.000000,0.000000,0.000000
3,"Women deserve to be abused, I guess.",0.666667,0.166667,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Women are made for making babies and cooking d...,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...
428,All people are fucked up because of these quee...,0.000000,0.000000,0.571429,0.000000,0.000000,0.000000,0.000000,0.571429
429,Actually THIS WEIRD PEOPLE are HARMFUL! They n...,0.000000,1.000000,0.000000,0.000000,0.333333,0.666667,0.000000,0.000000
430,ABU DA BI ABA DU BA BA HAMUD! See? I’m an arab...,0.000000,0.000000,0.000000,0.285714,0.000000,0.000000,0.857143,0.000000
431,a homo? so you fuck your brother instead of yo...,0.000000,0.666667,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000


In [7]:
df.duplicated

<bound method DataFrame.duplicated of                                                comment  violence  \
0            You should know women's sports are a joke  0.000000   
1      You look like Sloth with deeper Down’s syndrome  0.000000   
2    You look like Russian and speak like Indian. B...  1.000000   
3                 Women deserve to be abused, I guess.  0.666667   
4    Women are made for making babies and cooking d...  0.000000   
..                                                 ...       ...   
428  All people are fucked up because of these quee...  0.000000   
429  Actually THIS WEIRD PEOPLE are HARMFUL! They n...  0.000000   
430  ABU DA BI ABA DU BA BA HAMUD! See? I’m an arab...  0.000000   
431  a homo? so you fuck your brother instead of yo...  0.000000   
432  (((They))) will be chased out of every white c...  0.333333   

     directed_vs_generalized    gender      race  national_origin  disability  \
0                   0.000000  1.000000  0.000000         0.00000

In [8]:
df.isna().sum()

comment                    0
violence                   0
directed_vs_generalized    0
gender                     0
race                       0
national_origin            0
disability                 0
religion                   0
sexual_orientation         0
dtype: int64

In [9]:
# Initialize BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = TFBertModel.from_pretrained('bert-base-uncased')

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [22]:
# Tokenize input text
input_ids = []
for text in df['comment']:
    input_ids.append(tokenizer.encode(text, add_special_tokens=True, max_length=512, pad_to_max_length=True, truncation=True))


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2606: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [23]:
# Process input in batches
batch_size = 8
bert_outputs = []
for i in range(0, len(input_ids), batch_size):
    batch_input_ids = np.array(input_ids[i:i + batch_size])
    batch_bert_output = bert_model(batch_input_ids)[0]
    bert_outputs.append(batch_bert_output)

In [24]:
# Concatenate batch outputs
bert_output = np.concatenate(bert_outputs, axis=0)

In [25]:
# Convert labels to numpy array
labels = df[['violence',	'directed_vs_generalized',	'gender',	'race',	'national_origin',	'disability',	'religion', 'sexual_orientation']].values


In [26]:
# Ensure input data (bert_output) and labels have correct shapes
print("Shape of bert_output:", bert_output.shape)
print("Shape of labels:", labels.shape)

Shape of bert_output: (433, 512, 768)
Shape of labels: (433, 8)


In [27]:
# Split the data into training and testing sets (80% training, 20% testing)
train_data, test_data, train_labels, test_labels = train_test_split(bert_output, labels, test_size=0.2, random_state=42)


In [28]:
# Build neural network model for multi-label classification
input_layer = Input(shape=(train_data.shape[1], train_data.shape[2]), dtype=tf.float32)
dense_layer = Dense(128, activation='relu')(input_layer)
dropout_layer = Dropout(0.5)(dense_layer)
output_layer = Dense(7, activation='softmax')(dropout_layer)  # Softmax for multi-label classification
model = Model(inputs=input_layer, outputs=output_layer)

In [29]:
# Compile the model with categorical cross-entropy loss and Adam optimizer
model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])


In [30]:
# Training
model.fit(train_data, train_labels, epochs=5, batch_size=1)

Epoch 1/5


ValueError: ignored

In [ ]:
# Evaluation
loss, accuracy = model.evaluate(test_data, test_labels)

In [ ]:
print(f'Test Loss: {loss}')
print(f'Test Accuracy: {accuracy}')